In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = (SparkSession.builder.appName("Analyzing the vocabulary of Pride and Prejudice.").getOrCreate())

In [4]:
spark.sparkContext.setLogLevel("WARN")

In [5]:
spark?

Type:        SparkSession
String form: <pyspark.sql.session.SparkSession object at 0x00000221454D9B40>
File:        c:\pyspark\spark-3.2.3-bin-hadoop3.2\python\pyspark\sql\session.py
Docstring:  
The entry point to programming Spark with the Dataset and DataFrame API.

A SparkSession can be used create :class:`DataFrame`, register :class:`DataFrame` as
tables, execute SQL over tables, cache tables, and read parquet files.
To create a :class:`SparkSession`, use the following builder pattern:

.. autoattribute:: builder
   :annotation:

Examples
--------
>>> spark = SparkSession.builder \
...     .master("local") \
...     .appName("Word Count") \
...     .config("spark.some.config.option", "some-value") \
...     .getOrCreate()

>>> from datetime import datetime
>>> from pyspark.sql import Row
>>> spark = SparkSession(sc)
>>> allTypes = sc.parallelize([Row(i=1, s="string", d=1.0, l=1,
...     b=True, list=[1, 2, 3], dict={"s": 0}, row=Row(a=1),
...     time=datetime(2014, 8, 1, 14, 1, 5

In [6]:
book = spark.read.text("./1342-0.txt")

In [7]:
# column_1 = book.select(book["value"])
# column_2 = book.select(book.value)

In [8]:
# column_1.show(10, truncate = 70)
# column_2.show(10, truncate = 70)

In [9]:
# book.show(15, truncate=100)

In [10]:
from pyspark.sql.functions import col, split, explode, lower, regexp_extract

In [11]:
# import pyspark.sql.functions as F

# used functions:
#     1. col
#     2. explode
#     3. lower
#     4. split
#     5. regexp_extract

In [12]:
lines = book.select(split(col("value"), " ").alias("line"))

In [13]:
#lines = book.select(split(book.value, " ").alias("line"))

In [14]:
#lines = book.select(split(book.value, " ").alias("line"))

In [15]:
lines.show(10)

+--------------------+
|                line|
+--------------------+
|[The, Project, Gu...|
|                  []|
|[This, eBook, is,...|
|[almost, no, rest...|
|[re-use, it, unde...|
|[with, this, eBoo...|
|                  []|
|                  []|
|[Title:, Pride, a...|
|                  []|
+--------------------+
only showing top 10 rows



In [16]:
words = lines.select(explode(col("line")).alias("book"))

In [17]:
words.show(5)

+---------+
|     book|
+---------+
|      The|
|  Project|
|Gutenberg|
|    EBook|
|       of|
+---------+
only showing top 5 rows



In [18]:
# convert to lowercase

lower_words = words.select(lower(col("book")).alias("lower_word"))

In [19]:
lower_words.show(15)

+----------+
|lower_word|
+----------+
|       the|
|   project|
| gutenberg|
|     ebook|
|        of|
|     pride|
|       and|
|prejudice,|
|        by|
|      jane|
|    austen|
|          |
|      this|
|     ebook|
|        is|
+----------+
only showing top 15 rows



In [20]:
#  remove characters and spaces

clean_words = lower_words.select(regexp_extract(col("lower_word"), "[a-z]+", 0).alias("clean_word"))

remove_spaces = clean_words.filter(col("clean_word") != "")

In [21]:
remove_spaces.show(15)

+----------+
|clean_word|
+----------+
|       the|
|   project|
| gutenberg|
|     ebook|
|        of|
|     pride|
|       and|
| prejudice|
|        by|
|      jane|
|    austen|
|      this|
|     ebook|
|        is|
|       for|
+----------+
only showing top 15 rows



In [22]:
removed_spaces = clean_words.filter(col("clean_word") != "")

In [23]:
removed_spaces.show(15)

+----------+
|clean_word|
+----------+
|       the|
|   project|
| gutenberg|
|     ebook|
|        of|
|     pride|
|       and|
| prejudice|
|        by|
|      jane|
|    austen|
|      this|
|     ebook|
|        is|
|       for|
+----------+
only showing top 15 rows



In [46]:
grouped_words = removed_spaces.groupby(col("clean_word")).count()

In [47]:
grouped_words.show()

+-------------+-----+
|   clean_word|count|
+-------------+-----+
|       online|    4|
|         some|  209|
|        still|   72|
|          few|   72|
|         hope|  122|
|        those|   60|
|     cautious|    4|
|    imitation|    1|
|          art|    3|
|      solaced|    1|
|       poetry|    2|
|    arguments|    5|
| premeditated|    1|
|      elevate|    1|
|       doubts|    2|
|    destitute|    1|
|    solemnity|    5|
|   lieutenant|    1|
|gratification|    1|
|    connected|   14|
+-------------+-----+
only showing top 20 rows



In [55]:
grouped_words.orderBy(col("count").desc()).show(5)

+----------+-----+
|clean_word|count|
+----------+-----+
|       the| 4496|
|        to| 4235|
|        of| 3719|
|       and| 3602|
|       her| 2223|
+----------+-----+
only showing top 5 rows



In [75]:
results = grouped_words.orderBy(col("count"))

In [76]:
results.show(5)

+-------------+-----+
|   clean_word|count|
+-------------+-----+
| premeditated|    1|
|gratification|    1|
|      elevate|    1|
|   lieutenant|    1|
|    destitute|    1|
+-------------+-----+
only showing top 5 rows



In [88]:
# out_path = "./data/simple_count_single_partition.csv"

# results.write.format('csv').save("./data/my_file.csv")

results.toPandas().to_csv('l2.csv')

In [89]:
results.write.format('csv').save("./data/my_file.csv")

Py4JJavaError: An error occurred while calling o239.save.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.jobAbortedError(QueryExecutionErrors.scala:496)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:261)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:186)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:113)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:111)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.executeCollect(commands.scala:125)
	at org.apache.spark.sql.execution.QueryExecution$$anonfun$eagerlyExecuteCommands$1.$anonfun$applyOrElse$1(QueryExecution.scala:97)
	at org.apache.spark.sql.execution.SQLExecution$.$anonfun$withNewExecutionId$5(SQLExecution.scala:103)
	